In [1]:
import sys
sys.path.append("/Users/SJp/Documents/project_local/VIB-LLM-SS/ml-summerschool-2025/topic-1_data-integration-and-llms/project")

import pandas as pd
import numpy as np
import importlib
import src.VIB_project_functions as FF
importlib.reload(FF)


global_path  = "/Users/SJp/Documents/project_local/VIB-LLM-SS/ml-summerschool-2025/topic-1_data-integration-and-llms/project"

 # Load data
cytokines = pd.read_csv(f"{global_path}/data/Cytokines.csv", header=0, index_col=0).transpose()
clinical = pd.read_csv(f"{global_path}/data/Clinical.csv", header=0, index_col=0).transpose()
proteomics = pd.read_csv(f"{global_path}/data/MyelomaCells_proteomics.csv", header=0, index_col=0).transpose()
drugresponse = pd.read_csv(f"{global_path}/data/DrugResponse.csv", header=0, index_col=0).transpose().dropna()
 
# make sure all the above dataframes have the same patients in the same order
common_patients = set(clinical.index) & set(cytokines.index) & set(proteomics.index) & set(drugresponse.index)
common_patients = sorted(list(common_patients))
clinical = clinical.loc[common_patients].sort_index()
cytokines = cytokines.loc[common_patients].sort_index()
proteomics = proteomics.loc[common_patients].sort_index()
drugresponse = drugresponse.loc[common_patients].sort_index()
clinical_numeric = pd.get_dummies(clinical)


drug_classes = pd.read_csv(f"{global_path}/data/drugresponse_classes.csv", index_col=0)

## ML model to predict the drug response

In [ ]:

import warnings
warnings.filterwarnings('ignore')

results, patient_explanations=FF.run_drug_classification(
    proteomics, cytokines, clinical_numeric, drug_classes, 
    output_dir=f"{global_path}/results")

Skipping Bendamustine_DNA-alkylating agent (class counts: Counter({np.int64(1): 35, np.int64(0): 4}))
Skipping Bortezomib_Proteasome Inhibitor regime (class counts: Counter({np.int64(0): 21, np.int64(1): 18}))
Skipping Carfilzomib_Proteasome Inhibitor regime (class counts: Counter({np.int64(2): 22, np.int64(1): 16, np.int64(0): 1}))
Skipping Cisplatin_Platinum-containing (class counts: Counter({np.int64(1): 30, np.int64(0): 8, np.int64(2): 1}))
Skipping Cyclophosphamide_DNA-alkylating agent (class counts: Counter({np.int64(1): 35, np.int64(0): 3, np.int64(2): 1}))
Skipping Cytarabine_Nucleoitide analog (class counts: Counter({np.int64(1): 30, np.int64(2): 8, np.int64(0): 1}))
Skipping Etoposide_Topoisomerase inhibitor (class counts: Counter({np.int64(1): 27, np.int64(2): 11, np.int64(0): 1}))
Skipping Melphalan_DNA-alkylating agent (class counts: Counter({np.int64(1): 31, np.int64(0): 7, np.int64(2): 1}))
Skipping Dexamethasone+Bortezomib+Cyclophosphamide_Proteasome Inhibitor regime (c

In [3]:
FF.save_patient_graphs_json(patient_explanations, results, clinical, outdir=f"{global_path}/results/patient_graphs_json")

Saved /Users/SJp/Documents/project_local/VIB-LLM-SS/ml-summerschool-2025/topic-1_data-integration-and-llms/project/results/patient_graphs_json/MM058_KG.json
Saved /Users/SJp/Documents/project_local/VIB-LLM-SS/ml-summerschool-2025/topic-1_data-integration-and-llms/project/results/patient_graphs_json/MM028_KG.json
Saved /Users/SJp/Documents/project_local/VIB-LLM-SS/ml-summerschool-2025/topic-1_data-integration-and-llms/project/results/patient_graphs_json/MM072_KG.json
Saved /Users/SJp/Documents/project_local/VIB-LLM-SS/ml-summerschool-2025/topic-1_data-integration-and-llms/project/results/patient_graphs_json/MM048_KG.json
Saved /Users/SJp/Documents/project_local/VIB-LLM-SS/ml-summerschool-2025/topic-1_data-integration-and-llms/project/results/patient_graphs_json/MM036_KG.json
Saved /Users/SJp/Documents/project_local/VIB-LLM-SS/ml-summerschool-2025/topic-1_data-integration-and-llms/project/results/patient_graphs_json/MM092_KG.json
Saved /Users/SJp/Documents/project_local/VIB-LLM-SS/ml-sum

## LLM empowered results integration

### Generate the summary report of drug response prediction results including the feature SHAP

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from typing import Optional, List, Union
from pydantic import BaseModel, Field, field_validator
from typing import List, Optional
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import PydanticOutputParser
import json
import os
import re
import glob
import json
from json import JSONEncoder


In [ ]:
# Pick via provider:model string → works across providers
llm = init_chat_model(model="gemini-2.5-flash",
                      model_provider="google_genai",
                      temperature=0.2)


prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a biomedical-AI assistant that interprets predictions from a AI-powered predictive model for clinicians.
You are given a JSON with some information regarding the patient and 2 drugs. In the JSON, you are given:
- Patient ID
- Disease type
- For each drug, you will be given:
    - The drug name
    - The predicted class. This class can be one of three options: 
        - No effect if the drug is predicted as having no effect on treating the patient disease
        - Positive response if the drug is predicted as having a positive effect on treating the patient disease
        - Adverse effects if the drug is predicted as having a negative effect on the patient disease
    - Each predicted class has an associated probability
    - Each predicted class has associated features, that are responsible for the prediction. To reflect the importance of these features on the prediction we have the SHAP values. We have the top positive SHAP values and the top negative SHAP values.

Taking into account this JSON and the information explained above, I want you as a smart biomedical-AI assistant to pick the best of the two drugs. I don't want you to talk about the accuracy of the predicition for more than one sentence.
Once you have  picked the best drug for my patient, I want you to write a small report on the chosen drug, please include both positive an negative points about the drug, make it as straigthforward as possible (a maximum of 10 bullet points in total), and targeted towards clinicians.
After that, please write a short paragraph about the features involved in the decision making process, and look in the litterature for information about the relationship between these features and the disease the patient has."""),
    ("human", "{JSON_input}")
])

In [ ]:
# Path to your JSON file (e.g., patient PKG or classification output)
json_path = f"{global_path}/results/MM082.json"

with open(json_path, "r") as f:
    patient_json = json.load(f)

In [ ]:
chain = prompt | llm | StrOutputParser()
response = chain.invoke({"JSON_input": patient_json})
print(response)

class model_prediction_report(BaseModel):
    patient_ID: Optional[Union[str, int]]  # "555-1234", 5551234, or None
    disease_type: str = Field(description="e.g., Melanoma")
    recomended_drug_name: str = Field(description = "e.g., Pembrolizumab")
    info_on_recommended_drug: str = Field(description = "e.g., Pembrolizumab is a PD-1 inhibitor that has demonstrated significant efficacy in advanced melanoma, improving both progression-free and overall survival. Clinically, it can induce durable responses in a subset of patients. However, its use is associated with immune-related adverse effects, including colitis, hepatitis, pneumonitis, endocrinopathies (such as hypothyroidism or hypophysitis), and less commonly severe dermatologic or neurologic toxicities. Careful monitoring and prompt management of these toxicities are essential during treatment")
    decision_making_process: str 

parser = PydanticOutputParser(pydantic_object=model_prediction_report)
format_instructions = parser.get_format_instructions()
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract per schema:\n{format_instructions}"),
    ("human", "{text}"),
]).partial(format_instructions=format_instructions)

parsing_llm = prompt | llm | parser

# if `drug_text` is an AIMessage, use .content; otherwise pass the raw string
result = parsing_llm.invoke({"text": response})
print(result)

class MyEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__
MyEncoder().encode(result)

with open(".json", "w") as f:
    json.dump(MyEncoder().encode(result), f, indent=2)

For patient MM082, the AI-powered predictive model suggests that the **Dexamethasone+Bortezomib+Pomalidomide (Proteasome Inhibitor regime)** is the most suitable option, as it is the only regimen predicted to have a positive therapeutic effect. The model's confidence in this prediction is 88.6%. The other evaluated drugs (Lenalidomide_Corticosteroid + IMID, Prednisone_Corticosteroid, and Thalidomide_Corticosteroid + IMID) were all predicted to have no effect.

---

### Clinical Report: Dexamethasone+Bortezomib+Pomalidomide for Patient MM082

**Predicted Outcome:** Positive therapeutic effect.

**Key Considerations for Clinicians:**

*   **Positive Indicators (Factors favoring response):**
    *   Elevated levels of **YY1** (Yin Yang 1) protein are associated with a positive response to this regimen. YY1 is a transcription factor often overexpressed in Multiple Myeloma (MM), promoting cell growth and survival.
    *   Higher expression of **RHOT2** (Miro2), involved in mitochondrial dyn

### Functional Analysis of SHAP Features by Predicted Drug Response Class

In [13]:
# notebook.ipynb
import json
from src.GSEA_bioassistant import get_llm_with_tools, get_prompt_chain, analyze_patient

# --- Load a patient JSON ---
with open(f"{global_path}/results/patient_res_jsons/MM082.json", "r") as f:
    patient_data = json.load(f)

patient_id = "MM082"

# --- Setup LLM + chain ---
llm_with_tools = get_llm_with_tools()
chain = get_prompt_chain(llm_with_tools)

# --- Analyze patient ---
results, summaries = analyze_patient(patient_data, patient_id, chain)

# --- Show results ---
for cls, summary in summaries.items():
    print(f"\n### Drugs Predicted as {cls}")
    print(summary)


/Users/SJp/Documents/project_local/VIB-LLM-SS/ml-summerschool-2025/topic-1_data-integration-and-llms/project/src/GSEA_bioassistant.py:90: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  enrichr_query({"gene_list": list(set(feats["positive"]))}) if feats["positive"] else None



### Drugs Predicted as no_effect
The SHAP features indicate distinct biological processes influencing the "no_effect" prediction.

**Positive SHAP features (supporting "no_effect")** are primarily associated with the **regulation of immune responses**, particularly involving T cells (e.g., positive regulation of T cell proliferation and activation, regulation of immune response). Other notable pathways include **cell-cell adhesion mediated by integrin**, **posttranscriptional regulation of gene expression**, and **protein-containing complex assembly**. Key genes in these processes include `CD3E`, `IL15`, `ICAM3`, `NUDT21`, and `RBM4B`. These pathways suggest a more controlled or modulated cellular state, potentially contributing to a "no_effect" outcome.

**Negative SHAP features (against "no_effect")** are strongly linked to robust **inflammatory and cytotoxic immune responses**. Prominent terms include **neutrophil degranulation**, **neutrophil activation involved in immune response

### Summarizing patient knowledge graph